In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

We use a module to define layers:

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self,x):
        '''Used to define the output of the network'''
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # .view simply performs a reshape
        x = x.view(-1, self.num_flat_features(x))
        
        # Apply our non-linear transformations. Note, no normalization in output layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


Having defined the forward operation, `backwards` is automatically defined using autograd.

In [14]:
params = list(net.parameters())
print(len(params)) # This is weights x 5 + biases x 5?
print(params[0].size())
print(params[1].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])


In [16]:
input = Variable(torch.randn(1, 1, 32, 32)) # Inputs are Variables
out = net(input)
print(out)

Variable containing:
-0.0568  0.0072  0.1513  0.0086  0.0479  0.1040  0.0948  0.0466  0.0769 -0.0075
[torch.FloatTensor of size 1x10]



There is no concept of a single example input: all inputs have a batch dimension

In [18]:
input = Variable(torch.randn(10, 1, 32, 32)) # Inputs are Variables
out = net(input)
print(out)

Variable containing:
-0.0821  0.0137  0.1391  0.0007  0.0658  0.1239  0.0689  0.0370  0.0831 -0.0250
-0.0773  0.0070  0.1366  0.0227  0.0387  0.1290  0.0935  0.0405  0.0729 -0.0360
-0.0569  0.0039  0.1267  0.0091  0.0591  0.0954  0.1011  0.0555  0.0697 -0.0183
-0.0508  0.0201  0.1243  0.0220  0.0768  0.1043  0.0924  0.0285  0.0521 -0.0309
-0.0694  0.0098  0.1533  0.0127  0.0704  0.1031  0.0841  0.0474  0.0415 -0.0287
-0.0797 -0.0142  0.1379  0.0330  0.0561  0.1094  0.0851  0.0510  0.0886 -0.0335
-0.0486  0.0196  0.1302 -0.0057  0.0273  0.1302  0.0970  0.0148  0.0596 -0.0436
-0.0627 -0.0180  0.1592  0.0039  0.0713  0.0990  0.1020  0.0516  0.0640 -0.0220
-0.0643  0.0059  0.1422  0.0164  0.1081  0.1144  0.0750  0.0265  0.0720 -0.0278
-0.0658 -0.0090  0.1215  0.0459  0.0720  0.1045  0.0986  0.0474  0.0757 -0.0014
[torch.FloatTensor of size 10x10]

